In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.4.0-alpha"

#!import ./config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

#pragma warning disable SKEXP0011, SKEXP0003, SKEXP0052

var kernelBuilder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
var memoryBuilder = new MemoryBuilder();

// Grab the locally stored credentials from the settings.json file. 
// Name the service as "davinci" — assuming that you're using one of the davinci completion models. 
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
{
    kernelBuilder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    memoryBuilder.WithAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", azureEndpoint, apiKey)
        .WithMemoryStore(new VolatileMemoryStore());
}
else
{
    kernelBuilder.AddOpenAIChatCompletion(model, apiKey, orgId);
    memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey, orgId)
        .WithMemoryStore(new VolatileMemoryStore());
}

var kernel = kernelBuilder.Build();
var memory = memoryBuilder.Build();

In [ ]:
const string memoryCollectionName = "Facts About Me";

await memory.SaveInformationAsync(
    collection: memoryCollectionName,
    id: "LinkedIn Bio",
    text: "I currently work in the hotel industry at the front desk. I won the best team player award.");

await memory.SaveInformationAsync(
    collection: memoryCollectionName,
    id: "LinkedIn History",
    text: "I have worked as a tourist operator for 8 years. I have also worked as a banking associate for 3 years.");

await memory.SaveInformationAsync(
    collection: memoryCollectionName,
    id: "Recent Facebook Post",
    text: "My new dog Trixie is the cutest thing you've ever seen. She's just 2 years old.");

await memory.SaveInformationAsync(
    collection: memoryCollectionName,
    id: "Old Facebook Post",
    text: "Can you believe the size of the trees in Yellowstone? They're huge! I'm so committed to forestry concerns.");

Console.WriteLine("Four GIGANTIC vectors were generated just now from those 4 pieces of text above.");

In [ ]:
var executionSettings = new OpenAIPromptExecutionSettings{ MaxTokens= 100, Temperature = 0.8, TopP= 1 };
var myFunction = kernel.CreateFunctionFromPrompt(@"
Tell me about me and {{$input}} in less than 70 characters.", executionSettings);
var result = await kernel.InvokeAsync(myFunction, new(){ ["input"] = "my work history" });
Console.WriteLine(result);

In [ ]:
#pragma warning disable SKEXP0003

string ask = "Tell me about me and my work history.";
var relatedMemory = "I know nothing.";
var counter = 0;

var memories = memory.SearchAsync(memoryCollectionName, ask, limit: 5, minRelevanceScore: 0.77);

await foreach (MemoryQueryResult memoryResult in memories)
{
    if (counter == 0) { relatedMemory = memoryResult.Metadata.Text; }
    Console.WriteLine($"Result {++counter}:\n  >> {memoryResult.Metadata.Id}\n  Text: {memoryResult.Metadata.Text}  Relevance: {memoryResult.Relevance}\n");
}

In [ ]:
var executionSettings = new OpenAIPromptExecutionSettings{ MaxTokens= 100, Temperature = 0.8, TopP= 1 };
var myFunction = kernel.CreateFunctionFromPrompt(@"
    {{$input}}
    Tell me about me and my work history in less than 70 characters.
    ",
    executionSettings);
var result = await kernel.InvokeAsync(myFunction, new(){ ["input"] = relatedMemory });
Console.WriteLine(result);